In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import holodeck as holo
from holodeck import single_sources, utils, plot, detstats
from holodeck.constants import YR
import holodeck.anisotropy as anis

import numpy as np
import healpy as hp
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import cmath

# Spherical Harmonic Decomposition/Angular Power Spectrum

$$     a_{\ell m} = \frac{4\pi}{N_\mathrm{pix}} \sum_{p=0}^{N_\mathrm{pix}-1} Y^*_{\ell m} (\gamma_p) f(\gamma_p) $$
$$ C_\ell = \frac{1}{2\ell + 1} \sum_{m=-\ell}^\ell |a_{\ell m} |^2 $$
\
$f = h_c^2 / d\Omega$ = healpix map values, $N=12N_\mathrm{side}^2$ = number of pixels

# Spherical Harmonic Conventions

$$ Y_{\ell m}(\theta, \phi) = \lambda_{\ell m} (\cos\theta)e^{i m \phi} $$
$$ \lambda_{\ell m} = \sqrt{\frac{2\ell + 1}{4\pi}\frac{(\ell-m)!}{(\ell+m)!}} P_{\ell m}(x) $$
where $x\equiv \cos\theta$ and $P_{\ell m}(x)$ are the Legendre Polynomials


In [ ]:
def calc_alm(ell, emm, func, nside):
    """ 
    
    func : 
        function of ipix or theta,phi
    """
    npix = hp.nside2npix(nside)
    theta, phi = hp.pix2ang(nside, np.arange(npix))
    Ylm = sp.special.sph_harm(emm, ell, phi, theta)
    integrand = np.sum(np.conjugate(Ylm) * func)
    alm = 4*np.pi/npix * integrand

    
    return alm

def calc_cl(ell, func, nside):
    sum = 0
    for emm in range(-ell, ell+1):
        alm = calc_alm(ell, emm, func, nside)
        sum += alm * np.conjugate(alm)
    cl = sum/(2*ell + 1)
    return cl

In [ ]:
nside = 8
lmax=8
npix = hp.nside2npix(8)
moll = np.random.rand(npix)*100 / hp.nside2pixarea(nside)

ells = np.arange(lmax+1)
cl_drk = np.zeros_like(ells)
alm_drk = []
for ii, ell in enumerate(ells):
    cl_drk[ii] = calc_cl(ell, moll, nside)
    for emm in range(-ell, ell+1):
        alm_drk.append(calc_alm(ell, emm, moll, nside))

cl_hpy, alm_hpy = hp.anafast(moll, alm=True, lmax=lmax)

In [ ]:
print(cl_hpy.shape, alm_hpy.shape, np.array(alm_drk).shape)

In [ ]:
plt.plot(np.arange(len(alm_hpy)), alm_hpy, '-', label='healpy')
plt.plot(np.arange(len(alm_drk)), alm_drk, '--', label='equation')
plt.legend()

In [ ]:
print(cl_drk)

In [ ]:
fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

ells = np.arange(9)
cl_drk = np.zeros_like(ells)

nside_colors=['tab:red', 'tab:blue', 'tab:purple']
for nn, nside in enumerate([8,16,32]):
    npix = hp.nside2npix(nside)
    moll = np.random.rand(npix)*100 / hp.nside2pixarea(nside)

    for ii, ell in enumerate(ells):
        cl_drk[ii] = calc_cl(ell, moll, nside)

    cl_hpy = hp.anafast(moll)

    ax.plot(ells, cl_drk, label=f'drake, ns{nside}', linestyle=':', color=nside_colors[nn], linewidth=3)
    ax.plot(ells, cl_hpy[:len(cl_drk)], label=f'anafast, ns{nside}', color=nside_colors[nn])

ax.legend()

In [ ]:
fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')

ells = np.arange(9)
cl_drk = np.zeros_like(ells)

nside_colors=['tab:red', 'tab:blue', 'tab:purple']
for nn, nside in enumerate([8,16,32]):
    npix = hp.nside2npix(nside)
    moll = np.random.rand(npix)*100 / hp.nside2pixarea(nside)

    for ii, ell in enumerate(ells):
        cl_drk[ii] = calc_cl(ell, moll, nside)

    cl_hpy = hp.anafast(moll)

    ydrk = cl_drk[1:]/cl_drk[0]
    print(cl_drk.shape)
    ax.plot(ells[1:], cl_drk[1:]/cl_drk[0], label=f'drake, ns{nside}', linestyle=':', color=nside_colors[nn])
    ax.plot(ells[1:], cl_hpy[1:len(cl_drk)]/cl_hpy[0], label=f'anafast, ns{nside}', color=nside_colors[nn])

ax.legend()

# Nihan's Way

